# <p style="text-align:center;">Classification avec la méthode Random Forest</p>

## Import des bibliothèques nescessaires 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV

## Chargement des données en Data Frame

In [2]:
# Données d'entrainement
train_home = pd.read_csv('Datas/X_Train_Data/train_home_team_statistics_df.csv')
train_away = pd.read_csv('Datas/X_Train_Data/train_away_team_statistics_df.csv')
y_train = pd.read_csv('Datas/Y_train_1rknArQ.csv')

# Données de test
test_home = pd.read_csv('Datas/X_Test_Data/test_home_team_statistics_df.csv')
test_away = pd.read_csv('Datas/X_Test_Data/test_away_team_statistics_df.csv')
y_test = pd.read_csv('Datas/Y_test_random_sEE2QeA.csv')



## Nettoyage des données

In [3]:

# On enlève les deux premières colonnes qui n'ajoutent pas d'information
train_home_clean = train_home.drop(columns=["LEAGUE", "TEAM_NAME"])
train_away_clean = train_away.drop(columns=["LEAGUE", "TEAM_NAME"])


# On préfixe le nom de nos colonnes afin qu'elles ne se confondent pas
train_home_clean.columns = ["HOME_" + col for col in train_home_clean.columns]
train_away_clean.columns = ["AWAY_" + col for col in train_away_clean.columns]

test_home.columns = ["HOME_" + col for col in test_home.columns]
test_away.columns = ["AWAY_" + col for col in test_away.columns]

# Vérifie si les colonnes ont déjà été préfixées

# On fusionne nos données pour créer un unique data frame
X_train = pd.concat([train_home_clean, train_away_clean], axis=1)
X_test = pd.concat([test_home, test_away], axis=1)


# On traite les valeurs NaN en les remplacant par la moyenne de la colonne
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())


## Création des labels

In [4]:
# Ici on transforme nos 3 colonne binaire en une colonne non binaire. Cette colonne représentera nos labels.
# On donne la valeur 0 pour une victoire en invité, 1 pour une égalité eet 2 pour une victoire à domicile.

y_labels = y_train[["HOME_WINS", "DRAW", "AWAY_WINS"]].idxmax(axis=1).map({
    "AWAY_WINS": 0,
    "DRAW": 1,
    "HOME_WINS": 2
})

y_test_labels = y_test[["HOME_WINS", "DRAW", "AWAY_WINS"]].idxmax(axis=1).map({
    "AWAY_WINS": 0,
    "DRAW": 1,
    "HOME_WINS": 2
})

y_labels.head(10)

0    0
1    1
2    0
3    2
4    1
5    2
6    2
7    1
8    0
9    2
dtype: int64

## Choix du modèle

### Random Forest c'est quoi ? 
Crée une foret d'arbre de décision. Chacun des arbre qui la compose est entrainé sur un sous ensemble aléatoire du dataset.
Puis effectue une réduction sur l'ensemble des output des arbres de la foret (moyenne, moyenne entière)

### Pourquoi à t-on utilisé un Random Forest ?
Limite l'overfiting en comparaison à un modèle d'arbre de decision classique -> se généralise mieux



In [5]:

# model = GradientBoostingClassifier()
model = RandomForestClassifier(random_state=42, n_estimators=1000, max_depth=20, min_samples_split=2, criterion="gini" )
# n_estimators : nombre d'arbre qui compose la foret
# max_depth : max de question que peut poser un arbre -> limite le surapprentissage
# min_sample_split : Nombre minimum d’échantillons nécessaires pour diviser un nœud
# criterion : On utilise ici Gini car il a de meilleur performance sur le dataset de Test mais on aurait pu utiliser l'entropie


## Entrainement du modèle

In [6]:
model.fit(X_train, y_labels) 

RandomForestClassifier(max_depth=20, n_estimators=1000, random_state=42)

## Prédiction

In [7]:
y_pred = model.predict(X_test)

## Formatage de la prédiction pour benchmark

In [8]:
# On repasse de notre prédiciton sous forme de label a trois colonne binaires (Le meme format que y_train).

pred_df = pd.DataFrame(0, index=range(len(y_pred)), columns=["HOME_WINS", "DRAW", "AWAY_WINS"])
for i, pred in enumerate(y_pred):
    if pred == 2:
        pred_df.loc[i, "HOME_WINS"] = 1
    elif pred == 1:
        pred_df.loc[i, "DRAW"] = 1
    else:
        pred_df.loc[i, "AWAY_WINS"] = 1


pred_df["ID"] = y_test["ID"]
pred_df = pred_df[["ID", "HOME_WINS", "DRAW", "AWAY_WINS"]].sort_values("ID").reset_index(drop=True)

## Export CSV

In [9]:
# On exporte notre df en fichier CSV afin de pouvoir le déposer sur le site du Hackathon.

pred_df.to_csv("match_predictions.csv", index=False)
print("Fichier 'match_predictions.csv' généré avec succès.")

Fichier 'match_predictions.csv' généré avec succès.


## Benchmarks

![Position de ce modèle sur le site de challenge Data](./position_pred.png)
